# GUI CODE

In [52]:
from tkinter import *
from PIL import Image, ImageTk

# To get the dialog box to open when required
from tkinter import filedialog

In [54]:
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np

#loading the model
model = load_model("Main_gender_predictor_2x2_with_15epochs.model")

#Giving path of haarcascade file to detect the face. You can give your haarcascade file here
cascade = cv2.CascadeClassifier("C:/Users/lenovo/Anaconda3/envs/new_env/Lib/site-packages/haar/haarcascade_frontalface_default.xml")

#defining classes
category = ['men','women']

In [55]:
#function for counting the person in the frame
def facecounter(img, m, f, size=0.5):
    ## convert image into gray scaled image
    RGB_to_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = cascade.detectMultiScale(RGB_to_gray, 1.1,5)
    
    
    ## iterating over faces
    for (a,b,c,d) in faces:
        cv2.rectangle(img, (a,b), (a+c,b+d), (0,255,0),3)

        croppedPicture = np.copy(img[b:b+d,a:a+c])

        ## preprocess the image according to our model
        resizedFace = cv2.resize(croppedPicture, (96,96))
        
        
        ## cv2.imshow("cropped image",resized_Face)
        resizedFace = resizedFace.astype("float") / 255.0
        resizedFace = img_to_array(resizedFace)
        resizedFace = np.expand_dims(resizedFace, axis=0)


        ## model prediction
        output = model.predict(resizedFace)[0]

        ## get label with max accuracy
        index = np.argmax(output)
        label = category[index]

        ## calculating count
        if label == "women":
            f = f+1
        else:
            m = m+1

    cv2.rectangle(img, (0,0),(300,30),(255,255,255),-1)
    cv2.putText(img, " woman = {},man = {} ".format(f,m),(0,15),
    cv2.FONT_HERSHEY_TRIPLEX,0.6,(255, 101, 125),1)
    cv2.putText(img, " faces detected = " + str(len(faces)),(10,30),
    cv2.FONT_HERSHEY_TRIPLEX,0.5,(0,0,0),1)

    return img

In [90]:
#creating 1st window with Tk(). You can give any variable name instead of mnthn_root.
mnthn_root=Tk()
mnthn_root.geometry("1000x700")
mnthn_root.minsize(1000,700)
mnthn_root.maxsize(1000,700)
mnthn_root.title('Gender Detection Application')
mnthn_root.configure(bg='Aquamarine')


#inserting image 
photo = PhotoImage(file="gender3.png")
picture = Label(image=photo,bg='Aquamarine')


#inserting text
text=Label(text="Gender Detection Application",padx='3',borderwidth=2,bg='PeachPuff',font='Castellar  18 ',relief='ridge')


#positioning image and label
picture.grid(row=2,column=1)
text.grid(row=2,column=3)

   
# 3rd button command function
def img():
     # open file dialog box to select image
    # The dialogue box has a title "select image"
    filename = filedialog.askopenfilename(title ='"pen')
    image = cv2.imread(filename) #path to image

    ## maintaining separate counters
    man = 0
    woman = 0

    cv2.imshow("Gender FaceCounter",facecounter(image,man,woman ))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    


# 2nd button command function
def cam():
    
    #this code will open webcam for live detection
    source = cv2.VideoCapture(0)

    while True:
        ret, frame = source.read()
        x = 0
        y = 0
        cv2.imshow("Live Facecount",facecounter(frame, x, y))
        if cv2.waitKey(1)& 0xFF == ord('q'): #q is the Exit Key
            break

    source.release()
    cv2.destroyAllWindows()


# 1st button command function
def new():
    mnthn_root.destroy()
    root=Tk()
    root.geometry("1000x700")
    root.minsize(1000,700)
    root.maxsize(1000,700)
    root.configure(bg='Aquamarine')
    root.title('Gender Detection Application')
    
    #Button 2nd and 3rd declaration with font size and its dimension 
    b2=Button(root,text="Open camera",font='Times 22',height= 2, width=15,command=cam,bg='#6495ED')
    b2.place(x=600,y=300)
    b3=Button(root,text="Select image",font='Times 22',height= 2, width=15,command=img,bg='#6495ED')
    b3.place(x=150,y=300)
    b4=Button(root,text="Close",font="Times 10",width=10,command=root.destroy,bg='#96C8A2')
    b4.place(x=475,y=550)
    
    
#1st button on main window
b1=Button(mnthn_root,text="Get started...",command=new,bg='#FFA6C9')
b1.place(x=650,y=325)

mainloop()